# Get Conditional Access Exclusions
----
## Description

> This notebook utilizes the Microsoft Graph powershell modules to retrieve each conditional access policy, get all exclusions, then grabs the ids for each user / group and queries AzureAD to retrieve the friendly name. Finally it generates a csv to be used as a watchlist. The code in this notebook does not necessarily need to be run in a notebook. For example it could be run on prem, or and azure automation runbook, if modified to use App Auth with a certificate.

## Step breakdown
----
> 1. First install the Microsoft Graph powershell module. (First Time Only)
> 2. Import the Microsoft Graph module
> 3. Connect to Microsoft Graph with required scopes (using Device Authentication)
> 4. Get all conditional access policies and assign to a variable
> 5. For each policy get all users / groups that are excluded and add to custom object array
> 6. Convert array to CSV
> 7. Upload CSV to Azure Blob Storage


### Step 1. Install Microsoft Graph Powershell Module
----

In [ ]:
Install-Module ("Microsoft.Graph.Identity.SignIns","Microsoft.Graph.Users","Az.Storage") -Force -Verbose

### Steps 2-3 

**Import Modules and Connect to Graph**

----

<div style="color: Red; background-color: Khaki; padding: 3px; font-size: 16px;">
To authenticate run the cell below, copy the code in the output, click the link in the output, paste code previously copied, and sign in.
</div>

In [ ]:
Import-Module ("Microsoft.Graph.Identity.SignIns","Microsoft.Graph.Users","Az.Storage") -Verbose

In [ ]:
Connect-MgGraph -UseDeviceAuthentication -Scopes ("Policy.Read.All","User.Read.All","Group.Read.All")

### Steps 4-6
**Get Conditional Access Policies, Get Exclusions, Retrieve User / Group Name, Create Array, Create CSV**

----

In [ ]:
$CAPs = Get-MgIdentityConditionalAccessPolicy
[System.Collections.ArrayList]$PolicyArray = @()

foreach ($Policy in $CAPs){
    foreach($GroupExclusion in $Policy.Conditions.Users.ExcludeGroups){
        $GetGroup = (Get-mggroup -groupid ($GroupExclusion).tostring())
        $PolicyObject = [PSCustomObject]@{
            PolicyName = $Policy.DisplayName
            PolicyId = $Policy.Id
            ExcludedGroups = $GetGroup.DisplayName
            ExcludedUsers = $null
        }
        $PolicyArray.Add($PolicyObject) | out-null
    }
    
    foreach($UserExclusion in $Policy.Conditions.Users.ExcludeUsers){
        $GetUser = Get-mguser -userid $UserExclusion
        $PolicyObject = [PSCustomObject]@{
            PolicyName = $Policy.DisplayName
            PolicyId = $Policy.Id
            ExcludedGroups = $null
            ExcludedUsers = $GetUser.UserPrincipalName
        }
        $PolicyArray.Add($PolicyObject) | out-null        
    
    }

}

$PolicyArray | export-csv .\ConditionalAccessPolicyExclusions.csv -NoTypeInformation

In [ ]:
# Optional cell when running interactively to verify csv creation
Import-Csv ".\ConditionalAccessPolicyExclusions.csv" | fl

### Step 7

**Enter Sas Toke, Upload CSV to Azure Blog Storage, and remove created csv for cleanup**

----

In [ ]:
$SasToken = Read-Host "Enter your Sas token obtained from your Azure storage account container"

In [ ]:
$AzStorage = New-AzStorageContext -StorageAccountName "generalsentinel" -SasToken "$SasToken"
Set-AzStorageBlobContent -File ".\ConditionalAccessPolicyExclusions.csv" -Container "sentinel" -Context $AzStorage -force

In [ ]:
Remove-Item "ConditionalAccessPolicyExclusions.csv"